In [3]:
import pandas as pd

df = pd.read_excel('/srv/image-results/Com_VoluntariosEstudios_Predicciones.xlsx',
     engine='openpyxl')
df.rename(columns = {'Cantidad de vino estimada por el modelo':'Prediccion',
                     'Código del voluntario ':'IdVol'}, inplace = True)
df.head(3)

,#,IdVol,Correo electónico,Fecha,Momento de consumo,Tipo de vino,Cantidad de vino,Tipo de vaso,Prediccion,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,1,1f08ce1a-e69d-4e31-90e7-8505786c0b3a,jatokmls@gmail.com,14-06-2021_19-16-42,cena,crianza,125,no_determinado,NaN,NaN,NaN,NaN
1,2,cfc2a9a8-cf03-461b-8e50-63d3b28c6366,pedrodelrosal@gmail.com,28-05-2021_20-27-42,cena,reserva,150,copa_borg,NaN,NaN,NaN,NaN
2,2,cfc2a9a8-cf03-461b-8e50-63d3b28c6366,pedrodelrosal@gmail.com,15-06-2021_20-04-49,cena,crianza,100,copa_cata,NaN,NaN,NaN,NaN


In [102]:
import os
import json
import numpy as np

from imgclas.data_utils import load_image
from imgclas import paths, plot_utils

import warnings
warnings.filterwarnings("ignore") # To ignore UndefinedMetricWarning: [Recall/Precision/F-Score] is ill-defined and being set to 0.0 in labels with no [true/predicted] samples.

# User parameters to set
TIMESTAMP = '2022-09-20_OnlyCuasi'           # timestamp of the model
# SPLIT_NAME = 'testLaRiojaICVVCopasAll'                   # dataset split to predict
SPLIT_NAME = 'voluntarios_estudios'
MODEL_NAME = 'final_model.h5'         # model to use to make the mediction

# Set the timestamp
paths.timestamp = TIMESTAMP

# Load back the predictions
pred_path = os.path.join(paths.get_predictions_dir(), '{}+{}.json'.format(MODEL_NAME, SPLIT_NAME))
with open(pred_path) as f:
    pred_dict = json.load(f)

In [4]:
from imgclas import test_utils
import sklearn.metrics as metrics
# https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b

y, yhat = np.array(pred_dict['true_value']), np.array(pred_dict['pred_value'])

mae = metrics.mean_absolute_error(y, yhat)
mse = metrics.mean_squared_error(y, yhat)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(y,yhat)

print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

MAE: 26.044144043235278
MSE: 1103.7015864744599
RMSE: 33.22200455232134
R-Squared: -0.02985636255967883


In [82]:
df_pred=pd.DataFrame(columns=['IdVol', 'Fecha', 'Prediccion'])
# creamos un dataframe con el valor predicho para cada id de voluntario
for i in range(len(pred_dict['filenames'])):
    vol_id=pred_dict['filenames'][i].split('/')[-1].split('_')[0]
    value=pred_dict['pred_value'][i]
    a=pred_dict['filenames'][i].split('/')[-1].split('.')[0].split('_')[-2]
    b=pred_dict['filenames'][i].split('/')[-1].split('.')[0].split('_')[-1]
    fecha=a+'_'+b
    df_pred=df_pred.append(pd.DataFrame({'IdVol':[vol_id], 'Prediccion':[value], 'Fecha': fecha}))


In [93]:
df.drop(columns = ['Prediccion',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'], inplace=True)

In [98]:
# join pandas dataframe on a certain column
df_total=df.merge(df_pred[['Prediccion', 'IdVol', 'Fecha']], how = 'right',
                left_on = ['IdVol', 'Fecha '], 
         right_on = ['IdVol', 'Fecha']).drop(columns = ['Fecha'])
df_total.head(3)

,#,IdVol,Correo electónico,Fecha,Momento de consumo,Tipo de vino,Cantidad de vino,Tipo de vaso,Prediccion
0,4,5c61f838-32bc-4faa-858d-a215981d0618,ialbuernediaz@gmail.com,02-06-2021_20-59-54,cena,crianza,125,copa_cata,80.059959
1,19,681e4e03-2ca8-4b7e-a8c6-dc324d392ba7,lobosjb@gmail.com,19-11-2021_14-26-50,almuerzo,crianza,100,chiqui,129.464508
2,4,5c61f838-32bc-4faa-858d-a215981d0618,ialbuernediaz@gmail.com,12-06-2021_12-29-24,almuerzo,crianza,100,copa_cata,134.999771


In [96]:
df_total.shape

(229, 8)

In [99]:
df_total.to_excel('/srv/image-results/Com_VoluntariosEstudios_Predicciones_Miriam.xlsx') 